In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:16pt;}
div.text_cell_render.rendered_html{font-size:16pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:16pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
table.dataframe{font-size:16px;}
</style>
"""))

# 1. tensorflow v2.xx에서 v1사용하기

In [4]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() # tensorflow v2 비활성화하고 v1활성화
import numpy as np
import pandas as pd

## Tensorflow 
- 데이터 흐름 그래프(tensor 객체의 흐름)을 사용하는 수치 계산 라이브러리
- 그래프는 node(연산)와 edge로 구성
- sess = tf.Session()을 이용해서 실행 환경
- sess.run()을 통해서 실행 결과를 확인

In [5]:
# 1단계 : tensor 정의
node1 = tf.constant('Hello, Tensorflow')
# 2단계 : 세션(연산을 실행하는 환경) 생성
sess = tf.Session()
# 3단계 : 실제 실행 및 출력
print(sess.run(node1))
print(sess.run(node1).decode())

b'Hello, Tensorflow'
Hello, Tensorflow


In [6]:
# 간단한 연산 tensor 그래프
# 1. 그래프 정의
node1 = tf.constant(10, dtype=tf.float16)
node2 = tf.constant(20, dtype=tf.float16)
node3 = tf.add(node1, node2)
# 2. 세션 생성
sess = tf.Session()
# 3. 세션 실행 & 결과
print(sess.run([node1, node2, node3]))

[10.0, 20.0, 30.0]


In [7]:
# 타입변경
import numpy as np
node1 = tf.constant(np.array([1,2,3]), dtype=tf.int16)
node2 = tf.cast(node1, dtype=tf.float32)
sess = tf.Session()
print(sess.run([node1, node2]))

[array([1, 2, 3], dtype=int16), array([1., 2., 3.], dtype=float32)]


In [8]:
# 평균값 계산 : tf.reduce_mean()
data = np.array([1.,2.,3.,4.])
m = tf.reduce_mean(data)
sess = tf.Session()
print(sess.run(m))

2.5


In [9]:
# tf.random_normal([size]) : 평균이 0이고 표준편차가 1인 난수 size개 발생
w = tf.random_normal([3]) 
sess = tf.Session()
sess.run(w)

array([-0.5745614 , -0.30919367, -0.3228126 ], dtype=float32)

In [10]:
#  변수 노드
w = tf.Variable(tf.random_normal([1]))
sess = tf.Session()
sess.run(tf.global_variables_initializer()) # 변수 초기화
sess.run(w)

array([-0.11854505], dtype=float32)

# 2 tensorflow v1을 이용한 회귀분석 구현
## 2.1 독립변수 x가 1개, 타겟(종속)변수 y가 1개

In [11]:
# tensor 그래프 정의
x = np.array([1,2,3])
y = np.array([4,5,6])
# weight와 bias
W = tf.Variable(tf.random.normal([1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')
# Hat, Hypothesis : 결과는 numpy 배열
H = W*x+b
# cost function (손실함수 : mse)
cost = tf.reduce_mean(tf.square(H-y))
'''
학습목적 : cost가 최소가 되는 W,b를 찾는 것
cost 함수는 2차함수이므로 곡선 그래프, 곡선위 미분값이 0이 되는 방향
(경사하강법 GradientDescent)
'''
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)
# 세션 생성
sess = tf.Session()
# W,b 초기화
sess.run(tf.global_variables_initializer())
# 6000번 학습 (v2에서의 fit함수)
for step in range(6001):
    _, cost_val, W_val, b_val = sess.run([train, cost, W, b])
    if step%300==0:
        print("{}번째 cost:{}, W:{}, b{}".format(step, cost_val, W_val, b_val))

0번째 cost:1.0829070806503296, W:[2.255421], b[0.3622495]
300번째 cost:0.24298377335071564, W:[1.5711356], b[1.7016745]
600번째 cost:0.057334188371896744, W:[1.2774327], b[2.3693304]
900번째 cost:0.013528597541153431, W:[1.134765], b[2.6936471]
1200번째 cost:0.0031921917106956244, W:[1.065463], b[2.8511875]
1500번째 cost:0.0007532339659519494, W:[1.0317991], b[2.9277134]
1800번째 cost:0.0001777392899384722, W:[1.0154469], b[2.9648855]
2100번째 cost:4.1937932110158727e-05, W:[1.0075032], b[2.9829433]
2400번째 cost:9.89651834970573e-06, W:[1.0036448], b[2.9917145]
2700번째 cost:2.3356490146397846e-06, W:[1.0017706], b[2.995975]
3000번째 cost:5.523609729607415e-07, W:[1.000861], b[2.9980428]
3300번째 cost:1.3093732320612617e-07, W:[1.0004194], b[2.9990468]
3600번째 cost:3.1506015574223056e-08, W:[1.000206], b[2.9995327]
3900번째 cost:7.737128271401161e-09, W:[1.0001013], b[2.999768]
4200번째 cost:2.065121407213155e-09, W:[1.0000521], b[2.99988]
4500번째 cost:3.5627559236139916e-10, W:[1.0000219], b[2.9999504]
4800번째 cos

In [14]:
# 최종적으로 나온 회귀식 H = W*X+b
W_, b_ = sess.run([W,b])

In [15]:
def predict(x):
    return W_[0] * x + b_[0]

In [16]:
predict(5)

8.000059485435486

## 2.2 predict을 위한 placeholder 이용
- placeholder : 외부에서 데이터를 입력받을 수 있는 노드

In [17]:
x = tf.placeholder(dtype=tf.float32)
H = 1*x + 1
sess = tf.Session()
sess.run([x, H], feed_dict={x : np.array([40,50])})

[array([40., 50.], dtype=float32), array([41., 51.], dtype=float32)]

In [18]:
# tensor 그래프 정의
x_data = np.array([1,2,3])
y_data = np.array([2,3,4])
# placeholder 설정(x,y)
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)
# weight와 bias
W = tf.Variable(tf.random.normal([1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')
# Hat, Hypothesis : 결과는 numpy 배열
H = W*x+b
# cost function (손실함수 : mse)
cost = tf.reduce_mean(tf.square(H-y))
'''
학습목적 : cost가 최소가 되는 W,b를 찾는 것
cost 함수는 2차함수이므로 곡선 그래프, 곡선위 미분값이 0이 되는 방향
(경사하강법 GradientDescent)
'''
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)
# 세션 생성
sess = tf.Session()
# W,b 초기화
sess.run(tf.global_variables_initializer())
# 6000번 학습 (v2에서의 fit함수)
for step in range(6001):
    _, cost_val, W_val, b_val = sess.run([train, cost, W, b],
                                        feed_dict={x:x_data,y:y_data})
    if step%300==0:
        print("{}번째 cost:{}, W:{}, b{}".format(step, cost_val, W_val, b_val))

0번째 cost:3.886437177658081, W:[0.81691366], b[-0.39030623]
300번째 cost:0.04096197336912155, W:[1.234499], b[0.46692872]
600번째 cost:0.009665378369390965, W:[1.1139095], b[0.74105704]
900번째 cost:0.002280621090903878, W:[1.0553321], b[0.87421715]
1200번째 cost:0.00053814408602193, W:[1.0268781], b[0.9388997]
1500번째 cost:0.00012698236969299614, W:[1.0130563], b[0.97032005]
1800번째 cost:2.9961467589600943e-05, W:[1.006342], b[0.98558295]
2100번째 cost:7.0700698415748775e-06, W:[1.0030808], b[0.99299663]
2400번째 cost:1.6687063180143014e-06, W:[1.0014968], b[0.9965977]
2700번째 cost:3.9424011788469215e-07, W:[1.0007275], b[0.99834627]
3000번째 cost:9.333805195410605e-08, W:[1.0003543], b[0.9991954]
3300번째 cost:2.218485484206667e-08, W:[1.0001726], b[0.9996077]
3600번째 cost:5.315731410604485e-09, W:[1.0000843], b[0.9998079]
3900번째 cost:1.3189378478273284e-09, W:[1.0000416], b[0.99990445]
4200번째 cost:3.0301330888882205e-10, W:[1.0000204], b[0.999954]
4500번째 cost:8.90973197997802e-11, W:[1.0000113], b[0.999

In [19]:
# 예측하기
sess.run(H, feed_dict={x:5})

array([6.0000243], dtype=float32)

## 2.3 scale이 다른 데이터들의 회귀분석 구현(scale 조성X)

In [20]:

x_data = np.array([1, 2, 5, 8, 10])
y_data = np.array([5, 15, 68, 80, 95])
# placeholder 설정
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)
# weight & bias
W = tf.Variable(tf.random.normal([1]))
b = tf.Variable(tf.random.normal([1]))
# Hypothesis
H = W * x + b
# cost (손실함수)
cost = tf.reduce_mean(tf.square(H-y))
# 경사하강법
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cost)
# Session & 변수 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# 학습
for step in range(1, 6001):
    _, cost_val = sess.run([train, cost], feed_dict={x:x_data, y:y_data})
    if step%300==0:
        print("{}/6000번째 cost:{}".format(step+1, cost_val))

301/6000번째 cost:120.54051208496094
601/6000번째 cost:79.60441589355469
901/6000번째 cost:79.24958801269531
1201/6000번째 cost:79.24283599853516
1501/6000번째 cost:79.23918914794922
1801/6000번째 cost:79.23566436767578
2101/6000번째 cost:79.23228454589844
2401/6000번째 cost:79.22904205322266
2701/6000번째 cost:79.22590637207031
3001/6000번째 cost:79.2228775024414
3301/6000번째 cost:79.21993255615234
3601/6000번째 cost:79.21708679199219
3901/6000번째 cost:79.21440124511719
4201/6000번째 cost:79.2117691040039
4501/6000번째 cost:79.209228515625
4801/6000번째 cost:79.206787109375
5101/6000번째 cost:79.20442962646484
5401/6000번째 cost:79.20216369628906
5701/6000번째 cost:79.19993591308594
6001/6000번째 cost:79.19783020019531


## 2.4 scale이 다른 데이터들의 회귀분석 구현(조정O)
### scale 조정방법 : 모든 데이터를 일정범위내로 조정
- normalization(정규화) : 모든 데이터를 0~1 사이로 조정
    normalization = n-min/max-min = MinMaxscaler
- standarization(표준화) : 데이터의 평균을 0, 표준편차를 1로 조정
    standarization = n-mean/std(표준편차) = sklearn.preprocessing.StandardScaler

In [23]:
# 라이브러리 쓰지 않고 정규화
x_data = np.array([1,2,5,8,10])
y_data = np.array([5,15,68,80,95])
norm_scaled_x_data = (x_data-x_data.min()) / (x_data.max() - x_data.min())
norm_scaled_y_data = (y_data-y_data.min()) / (y_data.max() - y_data.min())

array([0.        , 0.11111111, 0.44444444, 0.77777778, 1.        ])

In [25]:
# 라이브러리를 사용하여 정규화
x_data = np.array([1,2,5,8,10]).reshape(-1,1)
y_data = np.array([5,15,68,80,95]).reshape(-1,1)
from sklearn.preprocessing import MinMaxScaler
scaler_x = MinMaxScaler() #독립변수 x를 정규화 시킬 객체
norm_scaled_x_data = scaler_x.fit_transform(x_data)
scaler_y = MinMaxScaler() # 종속(타겟)변수 y를 정규화시킬 객체
norm_scaled_y_data = scaler_y.fit_transform(y_data)

array([[0.        ],
       [0.11111111],
       [0.7       ],
       [0.83333333],
       [1.        ]])

In [26]:
# 라이브러리를 쓰지않고 표준화 하기
x_data = np.array([1,2,5,8,10])
y_data = np.array([5,15,68,80,95])
stan_scaled_x_data = (x_data-x_data.mean()) / x_data.std()
stan_scaled_y_data = (y_data-y_data.mean()) / y_data.std()
np.column_stack([x_data, norm_scaled_x_data, stan_scaled_x_data])

array([[ 1.        ,  0.        , -1.22474487],
       [ 2.        ,  0.11111111, -0.93313895],
       [ 5.        ,  0.44444444, -0.05832118],
       [ 8.        ,  0.77777778,  0.81649658],
       [10.        ,  1.        ,  1.39970842]])

In [27]:
# 라이브러리를 사용하여 표준화
x_data = np.array([1,2,5,8,10]).reshape(-1,1)
y_data = np.array([5,15,68,80,95]).reshape(-1,1)
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
stan_scaled_x_data = scaler_x.fit_transform(x_data)
scaler_y = StandardScaler()
stan_scaled_y_data = scaler_y.fit_transform(y_data)

In [ ]:
# 스케일 조정된 데이터를 다시 복구 : inverse_transform() 이용
scaler_y.inverse_transform(stan_scaled_y_data)

In [33]:
x_data = np.array([1, 2, 5, 8, 10])
y_data = np.array([5, 15, 68, 80, 95])
# placeholder 설정
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)
# weight & bias
W = tf.Variable(tf.random.normal([1]))
b = tf.Variable(tf.random.normal([1]))
# Hypothesis
H = W * x + b
# cost (손실함수)
cost = tf.reduce_mean(tf.square(H-y))
# 경사하강법
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cost)
# Session & 변수 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# 학습
for step in range(1, 9001):
    _, cost_val = sess.run([train, cost], feed_dict={x:norm_scaled_x_data, y:norm_scaled_y_data})
    if step%300==0:
        print("{}/9000번째 cost:{}".format(step, cost_val))

300/9000번째 cost:2.920863389968872
600/9000번째 cost:2.532590866088867
900/9000번째 cost:2.1980412006378174
1200/9000번째 cost:1.9097535610198975
1500/9000번째 cost:1.6613037586212158
1800/9000번째 cost:1.4471595287322998
2100/9000번째 cost:1.2625552415847778
2400/9000번째 cost:1.1033923625946045
2700/9000번째 cost:0.966137707233429
3000/9000번째 cost:0.8477497100830078
3300/9000번째 cost:0.7456099390983582
3600/9000번째 cost:0.6574634909629822
3900/9000번째 cost:0.5813690423965454
4200/9000번째 cost:0.5156546831130981
4500/9000번째 cost:0.4588806629180908
4800/9000번째 cost:0.4098075330257416
5100/9000번째 cost:0.3673678934574127
5400/9000번째 cost:0.33064261078834534
5700/9000번째 cost:0.2988402247428894
6000/9000번째 cost:0.2712792754173279
6300/9000번째 cost:0.2473728209733963
6600/9000번째 cost:0.22661557793617249
6900/9000번째 cost:0.20857223868370056
7200/9000번째 cost:0.19286838173866272
7500/9000번째 cost:0.17918092012405396
7800/9000번째 cost:0.1672321856021881
8100/9000번째 cost:0.15678277611732483
8400/9000번째 cost:0.147626608